In [1]:
import glob

In [2]:
# sample https://github.com/TylerKirby/latin-pos-tagging/blob/master/texts/perseus/phi0448.phi001.perseus-lat1.tb.txt
paths = glob.glob('./texts/perseus/*.txt')
print(len(paths))
word_list = []
for p in paths:
    #sample_text_path = "./texts/perseus/phi0448.phi001.perseus-lat1.tb.txt"
    file = open(p)
    text = file.read()
    text.replace('?', '.')
    text.replace('!', '.')
    text.strip('\n')
    sentence_list = text.split('.')
    strip = {'',',',';'}
    for sent in sentence_list:
        words = sent.split(' ')
        tuple_list = []
        for word in words:
            word_tag_pair = word.split('/')
            if (len(word_tag_pair) == 2) and (word_tag_pair[0] not in strip):
                tuple_list.append( (word_tag_pair[0], word_tag_pair[1]) )
        if len(tuple_list) > 0:
            word_list.append(tuple_list)

12


In [3]:
print(len(word_list))

3434


In [4]:
print(word_list)

[[('Omnis', 'a-p---ma-'), ('homines', 'n-p---ma-'), ('qui', 'p-p---mn-'), ('sese', 'p-p---ma-'), ('student', 'v3ppia---'), ('praestare', 'v--pna---'), ('ceteris', 'a-p---nd-'), ('animalibus', 'n-p---nd-'), ('summa', 'a-s---fb-'), ('ope', 'n-s---fb-'), ('niti', 'v--pnp---'), ('decet', 'v3spia---'), ('ne', 'c--------'), ('vitam', 'n-s---fa-'), ('silentio', 'n-s---nb-'), ('transeant', 'v3ppsa---'), ('veluti', 'd--------'), ('pecora', 'n-p---nn-'), ('quae', 'p-p---na-'), ('natura', 'n-s---fn-'), ('prona', 'a-p---na-'), ('atque', 'c--------'), ('ventri', 'n-s---md-'), ('oboedientia', 'v-pppana-'), ('finxit', 'v3sria---')], [('[0]', ''), ('\nsed', 'c--------'), ('nostra', 'a-s---fn-'), ('omnis', 'a-s---fn-'), ('vis', 'n-s---fn-'), ('in', 'r--------'), ('animo', 'n-s---mb-'), ('et', 'c--------'), ('corpore', 'n-s---nb-'), ('sita', 'v-srppfn-'), ('est', 'v3spia---'), (':', 'u--------'), ('\nanimi', 'n-s---mg-'), ('imperio', 'n-s---nb-'), ('corporis', 'n-s---ng-'), ('servitio', 'n-s---nb-'), ('

In [6]:
def features(sentence, index):
    """ sentence: [w1, w2, ...], index: the index of the word """
    return {
        'word': sentence[index],
        'is_first': index == 0,
        'is_last': index == len(sentence) - 1,
        'is_capitalized': sentence[index][0].upper() == sentence[index][0],
        'is_all_caps': sentence[index].upper() == sentence[index],
        'is_all_lower': sentence[index].lower() == sentence[index],
        'prefix-1': sentence[index][0],
        'prefix-2': sentence[index][:2],
        'prefix-3': sentence[index][:3],
        'suffix-1': sentence[index][-1],
        'suffix-2': sentence[index][-2:],
        'suffix-3': sentence[index][-3:],
        'prev_word': '' if index == 0 else sentence[index - 1],
        'next_word': '' if index == len(sentence) - 1 else sentence[index + 1],
        'has_hyphen': '-' in sentence[index],
        'is_numeric': sentence[index].isdigit(),
        'capitals_inside': sentence[index][1:].lower() != sentence[index][1:]
    }

In [7]:
def untag(tagged_sentence):
    return [w for w, t in tagged_sentence]

In [8]:
train_size = int(.75*len(word_list))
train_sent = word_list[:train_size]
test_sent = word_list[train_size:]
def transform_to_dataset(tagged_sentences):
    X, y = [], []
 
    for tagged in tagged_sentences:
        for index in range(len(tagged)):
            #print(tagged)
            X.append(features(untag(tagged), index))
            y.append(tagged[index][1])
 
    return X, y
 
X, y = transform_to_dataset(train_sent)
print(len(X))

48961


In [10]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline
 
clf = Pipeline([
    ('vectorizer', DictVectorizer(sparse=False)),
    ('classifier', DecisionTreeClassifier(criterion='entropy'))
])
 
clf.fit(X[:20000], y[:20000])   # Use only the first 10K samples if you're running it multiple times. It takes a fair bit :)
 
print ('Training completed')
 
X_test, y_test = transform_to_dataset(test_sent)
 
print ("Accuracy:" + str(clf.score(X_test, y_test)))

Training completed
Accuracy:0.5955996131528046
